In [516]:
import pandas as pd 
import geopandas as gpd 
from shapely.geometry import Point
import numpy as np 

In [517]:
# Load Wilayas Features and calculate polygon centroids
wilayas_shp = '/home/ali/GeoData/LimAdmin/wilayas.shp'
wilayas_gdf = gpd.read_file(wilayas_shp)
wilayas_gdf.rename(columns={"CODE": "code_wil"}, inplace=True)
wilayas_gdf['centroid'] = wilayas_gdf['geometry'].centroid
wilayas_gdf.head()

,WILAYA,code_wil,geometry,centroid
0,CHLEF,2,"POLYGON ((1.55326 36.02990, 1.55325 36.02988, ...",POINT (1.25669 36.23826)
1,LAGHOUAT,3,"POLYGON ((2.88464 32.88137, 2.87848 32.87936, ...",POINT (2.72955 33.60757)
2,OUM EL BOUAGHI,4,"POLYGON ((7.47320 35.52005, 7.47227 35.52063, ...",POINT (7.06527 35.83646)
3,BATNA,5,"POLYGON ((6.10102 35.05459, 6.10090 35.05447, ...",POINT (5.87455 35.41969)
4,BEJAIA,6,"MULTIPOLYGON (((4.39109 36.24618, 4.39083 36.2...",POINT (4.83635 36.57705)


In [518]:
# Load Tagreted dataset and set to_check to False assuming all geocoded coordinates are OK 
angem_branches = pd.read_csv('./angem_branches.csv')
angem_branches['to_check'] = False
angem_branches.head()

,code,nom,type,adresses,e-mail,fax,tel,wilayas de rattachement,lat,lon,to_check
0,23,ANNABA,Antenne,Cité AADL 221 logs Bloc w24 S01 ET S02 sidi ac...,arannaba@angem.dz,038 47 93 12,038 47 93 12,"Annaba, El taref, Skikda, Souk Ahras, Guelma",36.878511,7.714610,False
1,7,BISKRA,Antenne,BLOC N°06 CITE 200/500(PRES DE LA MOSQUEE EL F...,arbiskra@angem.dz,033 51 53 17,033 51 53 17,"Biskra, El Oued, Khenchla, Tebessa",37.172745,10.033169,False
2,6,BEJAIA,Antenne,"BVD KRIM BELKACEM,CENTRE D'AFFAIRES BEJAIA",arbejaia@angem.dz,034 11 31 26,034 11 31 25,"Bejaia, Tizi Ouzou, BBA, Setif, Bouira",36.744744,5.047844,False
3,8,BECHAR,Antenne,Ex hôpital 600 lit Bechar,arbechar@angem.dz,049 23 83 12,049 23 83 13,"Bechar, Tindouf, Adrar,Naama, El Bayadh",31.623810,-2.216244,False
4,25,CONSTANTINE,Antenne,204 logements SOREM tour C2 Nouvelle Ville Ali...,arconstantine@angem.dz,031 74 74 60,031 74 74 64,"Constantine, Jijel, Mila, O E B, Batna",36.257037,6.583138,False


In [519]:
# Convert pandas DataFrame to GeoPandas 
angem_branches['geometry'] = angem_branches.apply(lambda row: Point(row.lon, row.lat), axis=1)
angem_gdf  = gpd.GeoDataFrame(angem_branches)
angem_gdf.head()

,code,nom,type,adresses,e-mail,fax,tel,wilayas de rattachement,lat,lon,to_check,geometry
0,23,ANNABA,Antenne,Cité AADL 221 logs Bloc w24 S01 ET S02 sidi ac...,arannaba@angem.dz,038 47 93 12,038 47 93 12,"Annaba, El taref, Skikda, Souk Ahras, Guelma",36.878511,7.714610,False,POINT (7.71461 36.87851)
1,7,BISKRA,Antenne,BLOC N°06 CITE 200/500(PRES DE LA MOSQUEE EL F...,arbiskra@angem.dz,033 51 53 17,033 51 53 17,"Biskra, El Oued, Khenchla, Tebessa",37.172745,10.033169,False,POINT (10.03317 37.17275)
2,6,BEJAIA,Antenne,"BVD KRIM BELKACEM,CENTRE D'AFFAIRES BEJAIA",arbejaia@angem.dz,034 11 31 26,034 11 31 25,"Bejaia, Tizi Ouzou, BBA, Setif, Bouira",36.744744,5.047844,False,POINT (5.04784 36.74474)
3,8,BECHAR,Antenne,Ex hôpital 600 lit Bechar,arbechar@angem.dz,049 23 83 12,049 23 83 13,"Bechar, Tindouf, Adrar,Naama, El Bayadh",31.623810,-2.216244,False,POINT (-2.21624 31.62381)
4,25,CONSTANTINE,Antenne,204 logements SOREM tour C2 Nouvelle Ville Ali...,arconstantine@angem.dz,031 74 74 60,031 74 74 64,"Constantine, Jijel, Mila, O E B, Batna",36.257037,6.583138,False,POINT (6.58314 36.25704)


In [520]:
# Set the Coordinate reference system
angem_gdf = angem_gdf.set_crs("EPSG:4326")
wilayas_gdf = wilayas_gdf.set_crs("EPSG:4326")
angem_gdf.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [521]:
# Check for rows with no coordiantes
no_coords_gdf = angem_gdf[(angem_gdf['lat'].isnull() | angem_gdf['lat'].isnull())]

# Merge with Wialyas DataFrame and set geometry to centroids
no_coords_gdf = no_coords_gdf.merge(wilayas_gdf, left_on='code', right_on='code_wil')
no_coords_gdf['geometry'] = with_coords_gdf['centroid']
no_coords_gdf['to_check'] = True

with_coords_gdf = no_coords_gdf.drop(['geometry_x'], axis=1)

with_coords_gdf 

,code,nom,type,adresses,e-mail,fax,tel,wilayas de rattachement,lat,lon,to_check,WILAYA,code_wil,geometry_y,centroid,geometry
0,5,BATNA,Agence,CITE NACER CITE ADMINISTRATIVE BLOC 02 BATNA,batna@angem.dz,033 25 39 79,033 25 39 78,BATNA,NaN,NaN,True,BATNA,5,"POLYGON ((6.10102 35.05459, 6.10090 35.05447, ...",POINT (5.87455 35.41969),POINT (5.87455 35.41969)
1,20,SAIDA,Agence,BOULVARD FRANZ FANON EX MOHAFADAT DE SAIDA,saida@angem.dz,048 41 02 53,048 41 00 94,SAIDA,NaN,NaN,True,SAIDA,20,"POLYGON ((0.72323 34.44284, 0.72288 34.44221, ...",POINT (0.28119 34.73743),POINT (0.28119 34.73743)
2,29,MASCARA,Agence,RUE CHEMMOUM MOKHTAR MASCARA,mascara@angem.dz,045 71 89 98,045 71 89 36,MASCARA,NaN,NaN,True,MASCARA,29,"POLYGON ((0.15620 35.07143, 0.15508 35.07081, ...",POINT (0.15926 35.39167),POINT (0.15926 35.39167)
3,45,NAAMA,Agence,CITE ADMINISTRATIVE 50 BUREAUX BT 146 NAAMA,naama@angem.dz,049 59 50 09,049 59 56 63,NAAMA,NaN,NaN,True,NAAMA,45,"POLYGON ((-0.99792 32.51558, -0.99799 32.51650...",POINT (-0.76827 33.30472),POINT (-0.76827 33.30472)


In [522]:
# Update the Targeted DataFrame by adding Wilayas geometries and centroids 
angem_gdf = angem_gdf[(angem_gdf['lat'].notnull() & angem_gdf['lat'].notnull())]
angem_gdf = angem_gdf.merge(wilayas_gdf, left_on='code', right_on='code_wil')
angem_gdf.rename(columns={"geometry_x": "geometry"}, inplace=True)

# Conat the Targeted DataFrame with rows without coordinates and approximated with Wilayas centroids 
angem_gdf = pd.concat([angem_gdf, with_coords_gdf])
angem_gdf[(angem_gdf['lat'].isnull() | angem_gdf['lon'].isnull())]

,code,nom,type,adresses,e-mail,fax,tel,wilayas de rattachement,lat,lon,to_check,geometry,WILAYA,code_wil,geometry_y,centroid
0,5,BATNA,Agence,CITE NACER CITE ADMINISTRATIVE BLOC 02 BATNA,batna@angem.dz,033 25 39 79,033 25 39 78,BATNA,NaN,NaN,True,POINT (5.87455 35.41969),BATNA,5,"POLYGON ((6.10102 35.05459, 6.10090 35.05447, ...",POINT (5.87455 35.41969)
1,20,SAIDA,Agence,BOULVARD FRANZ FANON EX MOHAFADAT DE SAIDA,saida@angem.dz,048 41 02 53,048 41 00 94,SAIDA,NaN,NaN,True,POINT (0.28119 34.73743),SAIDA,20,"POLYGON ((0.72323 34.44284, 0.72288 34.44221, ...",POINT (0.28119 34.73743)
2,29,MASCARA,Agence,RUE CHEMMOUM MOKHTAR MASCARA,mascara@angem.dz,045 71 89 98,045 71 89 36,MASCARA,NaN,NaN,True,POINT (0.15926 35.39167),MASCARA,29,"POLYGON ((0.15620 35.07143, 0.15508 35.07081, ...",POINT (0.15926 35.39167)
3,45,NAAMA,Agence,CITE ADMINISTRATIVE 50 BUREAUX BT 146 NAAMA,naama@angem.dz,049 59 50 09,049 59 56 63,NAAMA,NaN,NaN,True,POINT (-0.76827 33.30472),NAAMA,45,"POLYGON ((-0.99792 32.51558, -0.99799 32.51650...",POINT (-0.76827 33.30472)


In [523]:
# Sort the DataFrame by code (Wilaya number) and reset the index
angem_gdf = angem_gdf.sort_values(by=['code'],ignore_index=True)
angem_gdf.reset_index(drop=True)
angem_gdf.head()

,code,nom,type,adresses,e-mail,fax,tel,wilayas de rattachement,lat,lon,to_check,geometry,WILAYA,code_wil,geometry_y,centroid
0,1,ADRAR,Agence,"CITE 500 VILLAS CNEP N°40, ADRAR",adrar@angem.dz,049 96 76 19,049 96 46 74,ADRAR,28.017440,-0.264250,False,POINT (-0.26425 28.01744),ADRAR,1,"POLYGON ((1.33461 20.72282, 1.32667 20.72491, ...",POINT (-0.62174 26.01400)
1,2,CHLEF,Agence,"RUE DJOUBA, HAY BEN SOUNA, CHLEF",chlef@angem.dz,027 79 21 59,027 79 21 58,CHLEF,36.160921,1.315595,False,POINT (1.31560 36.16092),CHLEF,2,"POLYGON ((1.55326 36.02990, 1.55325 36.02988, ...",POINT (1.25669 36.23826)
2,3,LAGHOUAT,Agence,AVENUE 05 JUILLET MAAMOURAH-LAGHOUAT,laghouat@angem.dz,029 13 25 37,029 13 25 38,LAGHOUAT,33.807834,2.862829,False,POINT (2.86283 33.80783),LAGHOUAT,3,"POLYGON ((2.88464 32.88137, 2.87848 32.87936, ...",POINT (2.72955 33.60757)
3,4,OEB,Agence,ANCIENNNE CITE ADMINISTRATIVE BP 2002 Oum El B...,oumelbouaghi@angem.dz,032 56 51 67,032 56 52 43,OEB,35.868879,7.110827,False,POINT (7.11083 35.86888),OUM EL BOUAGHI,4,"POLYGON ((7.47320 35.52005, 7.47227 35.52063, ...",POINT (7.06527 35.83646)
4,5,BATNA,Agence,CITE NACER CITE ADMINISTRATIVE BLOC 02 BATNA,batna@angem.dz,033 25 39 79,033 25 39 78,BATNA,NaN,NaN,True,POINT (5.87455 35.41969),BATNA,5,"POLYGON ((6.10102 35.05459, 6.10090 35.05447, ...",POINT (5.87455 35.41969)


In [524]:
# Check if coordinates are within Wilayas polygons
angem_gdf['coords_in_wil'] = [angem_gdf.geometry[i].within(angem_gdf.geometry_y[i]) for i in range(angem_gdf.geometry.count())] 
angem_gdf[angem_gdf.coords_in_wil == False]

,code,nom,type,adresses,e-mail,fax,tel,wilayas de rattachement,lat,lon,to_check,geometry,WILAYA,code_wil,geometry_y,centroid,coords_in_wil
7,7,BISKRA,Antenne,BLOC N°06 CITE 200/500(PRES DE LA MOSQUEE EL F...,arbiskra@angem.dz,033 51 53 17,033 51 53 17,"Biskra, El Oued, Khenchla, Tebessa",37.172745,10.033169,False,POINT (10.03317 37.17275),BISKRA,7,"POLYGON ((5.18354 34.41286, 5.18464 34.41222, ...",POINT (5.33496 34.46880),False
8,7,BISKRA,Agence,BLOC N° 06 CITE 200/500 (PRES DE LA MOSQUEE EL...,biskra@angem.dz,033 51 53 55,033 51 53 56,BISKRA,33.140374,5.595066,False,POINT (5.59507 33.14037),BISKRA,7,"POLYGON ((5.18354 34.41286, 5.18464 34.41222, ...",POINT (5.33496 34.46880),False
12,10,BOUIRA,Agence,CITE 56 LGTS BT F ENTREE 02 A COTE DU STADE MU...,bouira@angem.dz,026 72 02 72,026 72 02 72,BOUIRA,40.520423,22.971676,False,POINT (22.97168 40.52042),BOUIRA,10,"POLYGON ((3.77427 35.90221, 3.77327 35.90216, ...",POINT (3.84352 36.25275),False
42,33,ILLIZI,Agence,PLACE EL MAGHREB EL ARABI ILLIZI,illizi@angem.dz,029 41 16 10,029 41 16 10,ILLIZI,35.856345,-0.312907,False,POINT (-0.31291 35.85634),ILLIZI,33,"POLYGON ((7.71546 23.32467, 7.74599 23.35353, ...",POINT (8.32092 26.77824),False
50,41,SOUK AHRAS,Agence,CITE ADMINISTRATIVE 80 BUREAUX EL BORDJ EL KAB...,soukahras@angem.dz,037 72 43 86,037 72 43 87,SOUK AHRAS,35.515206,0.300470,False,POINT (0.30047 35.51521),SOUK-AHRAS,41,"POLYGON ((7.53798 35.88014, 7.53725 35.88022, ...",POINT (7.88354 36.14933),False
53,43,MILA,Agence,CITE BEN GUERBA FACE APW MILA,mila@angem.dz,031 47 48 16,031 47 47 44,MILA,47.255310,-122.472227,False,POINT (-122.47223 47.25531),MILA,43,"POLYGON ((6.01335 35.89529, 6.01206 35.89615, ...",POINT (6.14584 36.29332),False
56,46,AIN TEMOUCHENT,Agence,AVENUE AMRO MOKDAD CITE MEDINA,aintemouchent@angem.dz,043 79 53 34,043 79 21 38,AIN TEMOUCHENT,38.588127,-90.297416,False,POINT (-90.29742 38.58813),AIN TEMOUCHENT,46,"MULTIPOLYGON (((-1.06748 35.12752, -1.06791 35...",POINT (-1.05368 35.35008),False


In [525]:
# Update to_check flag to take into account points outside Wilays and without geocoding coordinates
final_gdf = angem_gdf
final_gdf['to_check'] = (angem_gdf['to_check'] == True) | (angem_gdf['coords_in_wil'] == False)   

# Remove unecessary columns
final_gdf = final_gdf.drop(['geometry_y', 'code_wil', 'centroid', 'WILAYA', 'coords_in_wil'], axis=1)

final_gdf[final_gdf.to_check == True]

,code,nom,type,adresses,e-mail,fax,tel,wilayas de rattachement,lat,lon,to_check,geometry
4,5,BATNA,Agence,CITE NACER CITE ADMINISTRATIVE BLOC 02 BATNA,batna@angem.dz,033 25 39 79,033 25 39 78,BATNA,NaN,NaN,True,POINT (5.87455 35.41969)
7,7,BISKRA,Antenne,BLOC N°06 CITE 200/500(PRES DE LA MOSQUEE EL F...,arbiskra@angem.dz,033 51 53 17,033 51 53 17,"Biskra, El Oued, Khenchla, Tebessa",37.172745,10.033169,True,POINT (10.03317 37.17275)
8,7,BISKRA,Agence,BLOC N° 06 CITE 200/500 (PRES DE LA MOSQUEE EL...,biskra@angem.dz,033 51 53 55,033 51 53 56,BISKRA,33.140374,5.595066,True,POINT (5.59507 33.14037)
12,10,BOUIRA,Agence,CITE 56 LGTS BT F ENTREE 02 A COTE DU STADE MU...,bouira@angem.dz,026 72 02 72,026 72 02 72,BOUIRA,40.520423,22.971676,True,POINT (22.97168 40.52042)
25,20,SAIDA,Agence,BOULVARD FRANZ FANON EX MOHAFADAT DE SAIDA,saida@angem.dz,048 41 02 53,048 41 00 94,SAIDA,NaN,NaN,True,POINT (0.28119 34.73743)
37,29,MASCARA,Agence,RUE CHEMMOUM MOKHTAR MASCARA,mascara@angem.dz,045 71 89 98,045 71 89 36,MASCARA,NaN,NaN,True,POINT (0.15926 35.39167)
42,33,ILLIZI,Agence,PLACE EL MAGHREB EL ARABI ILLIZI,illizi@angem.dz,029 41 16 10,029 41 16 10,ILLIZI,35.856345,-0.312907,True,POINT (-0.31291 35.85634)
50,41,SOUK AHRAS,Agence,CITE ADMINISTRATIVE 80 BUREAUX EL BORDJ EL KAB...,soukahras@angem.dz,037 72 43 86,037 72 43 87,SOUK AHRAS,35.515206,0.300470,True,POINT (0.30047 35.51521)
53,43,MILA,Agence,CITE BEN GUERBA FACE APW MILA,mila@angem.dz,031 47 48 16,031 47 47 44,MILA,47.255310,-122.472227,True,POINT (-122.47223 47.25531)
55,45,NAAMA,Agence,CITE ADMINISTRATIVE 50 BUREAUX BT 146 NAAMA,naama@angem.dz,049 59 50 09,049 59 56 63,NAAMA,NaN,NaN,True,POINT (-0.76827 33.30472)


In [528]:
# Write to a GeoPackage file

#gpd.GeoDataFrame(final_gdf).to_file("angem_to_check.geojson", driver='GeoJSON')